# Traffic Sign Recognition Model
## Environment Initialisation

In [ ]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")
!ls
import os
os.chdir("/content/drive/My Drive/[ Masters ] - Deep Learning Proj/Model")
!pip install import_ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [ ]:
# Libraries
import os
import sys
import tensorflow as tf
import numpy as np
import math
IntialiseEnv()
import import_ipynb
import pickle
from sklearn.model_selection import train_test_split

# from Data_Preparation.Data_Preparation import LoadTrainDataSetWithAugmentation
# from Data_Preparation.Data_Preparation import LoadTrainDataSet
# from Data_Preparation.Data_Preparation import LoadTestDataSet

## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [ ]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0
/device:GPU:0
Num GPUs Available:  1


## Reading DataSet
Pre-processing and Loading Train and Test DataSet respectively

In [ ]:
def ReadTrainDataSet():
    print("Reading Train Pre-processed DataSet")
    processedTrainDataSet = '/content/drive/My Drive/[ Masters ] - Deep Learning Proj/DataSet/Processed_DataSet/GermanTrainDataSet.pkl'
    isProcessedTrainDataSetExits= os.path.exists(processedTrainDataSet)
    train_image_array, train_image_labels = None, None
    
    if isProcessedTrainDataSetExits:
        print("Loading Processed Train DataSet from Processed_DataSet/TrainDataSet.pkl")
        file = open(processedTrainDataSet, 'rb')
        train_image_array, train_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Train DataSet.")
    else:
        print("Processed_DataSet/TrainDataSet.pkl file does not exist")
        print("Loading Train DataSet ... This may take a while.")
        train_image_array, train_image_labels =  LoadTrainDataSetWithAugmentation()
        file = open(processedTrainDataSet, 'wb')
        pickle.dump((train_image_array, train_image_labels), file, protocol=4)
        file.close()
        print("Saving pre-processed train DataSets in Processed_DataSet/TrainDataSet.pkl")
    return train_image_array, train_image_labels    

In [ ]:
def ReadTestDataSet():
    processedTestDataSet = '/content/drive/My Drive/[ Masters ] - Deep Learning Proj/DataSet/Processed_DataSet/GermanTestDataSet.pkl'
    isProcessedTestDataSetExits= os.path.exists(processedTestDataSet)
    test_image_array, test_image_labels = None, None
    
    if isProcessedTestDataSetExits:
        print("Loading Processed Test DataSet from Processed_DataSet/GermanTestDataSet.pkl")
        file = open(processedTestDataSet, 'rb')
        test_image_array, test_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Test DataSet.")
    else:
        print("Processed_DataSet/TestDataSet.pkl file does not exist")
        print("Loading Test DataSet ... This may take a while.")
        test_image_array, test_image_labels = LoadTestDataSet()
        file = open(processedTestDataSet, 'wb')
        pickle.dump((test_image_array, test_image_labels), file)
        file.close()
        print("Saving pre-processed test DataSets in Processed_DataSet/TestDataSet.pkl")
    return test_image_array, test_image_labels

In [ ]:
train_image_array, train_image_labels= ReadTrainDataSet()

Reading Train Pre-processed DataSet
Loading Processed Train DataSet from Processed_DataSet/TrainDataSet.pkl
Done Loading Train DataSet.


In [ ]:
print(train_image_array.shape)

(39209, 3600)


In [ ]:
print(train_image_array.shape)

(39209, 3600)


In [ ]:
test_image_array, test_image_labels= ReadTestDataSet()

Loading Processed Test DataSet from Processed_DataSet/GermanTestDataSet.pkl
Done Loading Test DataSet.


In [ ]:
print(test_image_array.shape)

(12630, 60, 60, 1)


## Train-Split 
Split Train images dataset into two splits: training and validation respectively 80-20

In [ ]:
image_train, image_valid,label_train, label_valid = train_test_split(train_image_array, train_image_labels,stratify=train_image_labels,test_size=0.1,random_state=43)

In [ ]:
print(image_train.shape)

(35288, 3600)


In [ ]:
del train_image_array
del train_image_labels

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,ELU
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D 
from keras.models import Sequential
from keras.engine.topology import Layer
class Residual(Layer):
    def __init__(self, channels_in,kernel,**kwargs):
        super(Residual, self).__init__(**kwargs)
        self.channels_in = channels_in
        self.kernel = kernel

    def call(self, x):
        # the residual block using Keras functional API
        first_layer =Activation("linear", trainable=False)(x)
        x = Conv2D(self.channels_in,self.kernel, padding="same")(first_layer)
        x = Activation("relu")(x)
        x = Conv2D(self.channels_in,self.kernel,padding="same")(x)
        residual= Add()([x, first_layer])
        x = Activation("relu")(residual)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape

## Hyper-parameters Tuning

In [ ]:
# Hyper-parameters tuning
kernel_2 = (3,3)
pooling = (2,2)
dropout = 0.2
num_classes = 43

## Model Architecture

In [ ]:
weight_decay=1E-4
from keras.regularizers import l2

def model_():
  
    model = None
    tf.initializers.Orthogonal(gain=1.0, seed=None)
    model=Sequential()


    model.add(Conv2D(32,kernel_2, input_shape=(60,60,1), strides = 1, padding='valid',activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    model.add(Conv2D(128,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    model.add(Conv2D(256,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(512, activation = tf.nn.relu))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation = 'softmax'))
    return model

## Model Optimizer 

In [ ]:
from keras.callbacks import Callback

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='val_acc', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [ ]:
# class BatchEarlyStopping(Callback):
#     """Stop training when a monitored quantity has stopped improving.
#     # Arguments
#         monitor: quantity to be monitored.
#         min_delta: minimum change in the monitored quantity
#             to qualify as an improvement, i.e. an absolute
#             change of less than min_delta, will count as no
#             improvement.
#         patience: number of batches with no improvement
#             after which training will be stopped.
#         verbose: verbosity mode.
#         mode: one of {auto, min, max}. In `min` mode,
#             training will stop when the quantity
#             monitored has stopped decreasing; in `max`
#             mode it will stop when the quantity
#             monitored has stopped increasing; in `auto`
#             mode, the direction is automatically inferred
#             from the name of the monitored quantity.
#         baseline: Baseline value for the monitored quantity to reach.
#             Training will stop if the model doesn't show improvement
#             over the baseline.
#         restore_best_weights: whether to restore model weights from
#             the batch with the best value of the monitored quantity.
#             If False, the model weights obtained at the last step of
#             training are used.
#     """

#     def __init__(self,
#                  monitor='acc',
#                  min_delta=0,
#                  patience=0,
#                  verbose=0,
#                  mode='auto',
#                  baseline=None,
#                  restore_best_weights=False):
#         super(BatchEarlyStopping, self).__init__()

#         self.monitor = monitor
#         self.baseline = baseline
#         self.patience = patience
#         self.verbose = verbose
#         self.min_delta = min_delta
#         self.wait = 0
#         self.stopped_batch = 0
#         self.restore_best_weights = restore_best_weights
#         self.best_weights = None

#         if mode not in ['auto', 'min', 'max']:
#             warnings.warn('BatchEarlyStopping mode %s is unknown, '
#                           'fallback to auto mode.' % mode,
#                           RuntimeWarning)
#             mode = 'auto'

#         if mode == 'min':
#             self.monitor_op = np.less
#         elif mode == 'max':
#             self.monitor_op = np.greater
#         else:
#             if 'acc' in self.monitor:
#                 self.monitor_op = np.greater
#             else:
#                 self.monitor_op = np.less

#         if self.monitor_op == np.greater:
#             self.min_delta *= 1
#         else:
#             self.min_delta *= -1

#     def on_train_begin(self, logs=None):
#         # Allow instances to be re-used
#         self.wait = 0
#         self.stopped_batch = 0
#         if self.baseline is not None:
#             self.best = self.baseline
#         else:
#             self.best = np.Inf if self.monitor_op == np.less else -np.Inf

#     def on_batch_end(self, batch, logs=None):
#         current = self.get_monitor_value(logs)
#         if current is None:
#             return

#         if self.monitor_op(current - self.min_delta, self.best):
#             self.best = current
#             self.wait = 0
#             if self.restore_best_weights:
#                 self.best_weights = self.model.get_weights()
#         else:
#             self.wait += 1
#             if self.wait >= self.patience:
#                 self.stopped_batch = batch
#                 self.model.stop_training = True
#                 if self.restore_best_weights:
#                     if self.verbose > 0:
#                         print('Restoring model weights from the end of '
#                               'the best batch')
#                     self.model.set_weights(self.best_weights)

#     def on_train_end(self, logs=None):
#         if self.stopped_batch > 0 and self.verbose > 0:
#             print('Batch %05d: early stopping' % (self.stopped_batch + 1))

#     def get_monitor_value(self, logs):
#         monitor_value = logs.get(self.monitor)
#         if monitor_value is None:
#             warnings.warn(
#                 'Early stopping conditioned on metric `%s` '
#                 'which is not available. Available metrics are: %s' %
#                 (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
#             )
#         return monitor_value

In [ ]:
class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss',
                 min_delta=0, patience=0, verbose=0, mode='auto',baseline=0.04):
        super(BatchEarlyStopping, self).__init__()
        self.monitor = monitor
        self.patience = patience
        self.verbose = verbose
        self.baseline = baseline
        self.min_delta = min_delta
        self.wait = 0
        self.stopped_batch = 0
        if mode not in ['auto', 'min', 'max']:
            warnings.warn('BatchEarlyStopping mode %s is unknown, '
                          'fallback to auto mode.' % mode,
                          RuntimeWarning)
            mode = 'auto'
        if mode == 'min':
            self.monitor_op = np.less
        elif mode == 'max':
            self.monitor_op = np.greater
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
            else:
                self.monitor_op = np.less
        if self.monitor_op == np.greater:
            self.min_delta *= 1
        else:
            self.min_delta *= -1
    def on_train_begin(self, logs=None):
      # Allow instances to be re-used
      self.wait = 0
      self.stopped_batch = 0
      if self.baseline is not None:
          self.best = self.baseline
      else:
          self.best = np.Inf if self.monitor_op == np.less else -np.Inf
    def on_batch_end(self, batch, logs=None):
        current = logs.get(self.monitor)
        # print('loss ',current,' for batch',batch)

        if current is None:
            warnings.warn(
                'Batch Early stopping conditioned on metric `%s` '
                'which is not available. Available metrics are: %s' %
                (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
            )
            return
        if self.monitor_op(current - self.min_delta, self.best):
            self.best = current
            self.wait = 0
        else:
          # print('loss',current)
          self.wait += 1
          if self.wait >= self.patience:
              self.stopped_batch = batch
              self.model.stop_training = True
    def on_train_end(self, logs=None):
        if self.stopped_batch > 0 and self.verbose > 0:
            print('Batch %05d: early stopping' % (self.stopped_batch + 1))

In [ ]:
from tensorflow import keras
model = model_()


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2,  verbose=1, min_delta=1e-4, min_lr=1e-20)
# optimizer = keras.optimizers.Adadelta()

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)

In [ ]:
x=aug.flow(image_train.reshape(image_train.shape[0],60,60,1), np.array(label_train), batch_size=32)

## Model Training

In [ ]:
import warnings

In [ ]:
history = model.fit(x, epochs=80, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/80
1103/1103 [==============================] - 22s 20ms/step - loss: 6.8855 - sparse_categorical_accuracy: 0.5443 - val_loss: 4.5429 - val_sparse_categorical_accuracy: 0.9174
Epoch 2/80
1103/1103 [==============================] - 22s 20ms/step - loss: 4.0268 - sparse_categorical_accuracy: 0.8239 - val_loss: 2.8935 - val_sparse_categorical_accuracy: 0.9719
Epoch 3/80
1103/1103 [==============================] - 22s 20ms/step - loss: 2.6023 - sparse_categorical_accuracy: 0.9001 - val_loss: 1.8989 - val_sparse_categorical_accuracy: 0.9834
Epoch 4/80
1103/1103 [==============================] - 22s 20ms/step - loss: 1.7284 - sparse_categorical_accuracy: 0.9345 - val_loss: 1.2735 - val_sparse_categorical_accuracy: 0.9872
Epoch 5/80
1103/1103 [==============================] - 22s 20ms/step - loss: 1.1866 - sparse_categorical_accuracy: 0.9478 - val_loss: 0.8730 - val_sparse_categorical_accuracy: 0.9880
Epoch 6/80
1103/1103 [==============================] - 22s 20ms/step - loss: 0.

In [ ]:
train_on_batch(x)

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0314 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0307 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0312 - sparse_categorical_accuracy: 0.9993 - val_loss: 0.0318 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0303 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0295 - val_sparse_categorical_accuracy: 0.9990
Epoch 4/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0299 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0301 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0294 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0298 - val_sparse_categorical_accuracy: 0.9992


In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0291 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0282 - val_sparse_categorical_accuracy: 0.9995
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0289 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0282 - val_sparse_categorical_accuracy: 0.9990
Epoch 3/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0285 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0275 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0283 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0274 - val_sparse_categorical_accuracy: 0.9995
Epoch 5/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0281 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0276 - val_sparse_categorical_accuracy: 0.9990


In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0273 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0277 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0278 - sparse_categorical_accuracy: 0.9993 - val_loss: 0.0263 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1102/1103 [============================>.] - ETA: 0s - loss: 0.0274 - sparse_categorical_accuracy: 0.9995
Epoch 00003: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0274 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0269 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0268 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0266 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.027

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0265 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0263 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0264 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0262 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0259 - sparse_categorical_accuracy: 0.9999 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0269 - sparse_categorical_accuracy: 0.9992 - val_loss: 0.0258 - val_sparse_categorical_accuracy: 0.9995
Epoch 5/5
1102/1103 [============================>.] - ETA: 0s - loss: 0.0265 - sparse_categorical_accuracy: 0.9995
Epoch 00005: ReduceLROnPlateau reducing learning rate to 1.599999814061448e-05.
1103/1103 [==============================] - 22s 20ms/step - loss: 0.026

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0261 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0263 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0259 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0263 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0265 - sparse_categorical_accuracy: 0.9994 - val_loss: 0.0262 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1102/1103 [============================>.] - ETA: 0s - loss: 0.0263 - sparse_categorical_accuracy: 0.9997
Epoch 00004: ReduceLROnPlateau reducing learning rate to 3.199999628122896e-06.
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0263 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0262 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.026

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0261 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1102/1103 [============================>.] - ETA: 0s - loss: 0.0262 - sparse_categorical_accuracy: 0.9997
Epoch 00003: ReduceLROnPlateau reducing learning rate to 6.399999165296323e-07.
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0262 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0257 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.026

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0259 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1101/1103 [============================>.] - ETA: 0s - loss: 0.0264 - sparse_categorical_accuracy: 0.9996
Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.2799998785339995e-07.
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0264 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0259 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1101/1103 [============================>.] - ETA: 0s - loss: 0.0261 - s

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0265 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1103/1103 [==============================] - ETA: 0s - loss: 0.0263 - sparse_categorical_accuracy: 0.9996
Epoch 00003: ReduceLROnPlateau reducing learning rate to 5.1199993578165965e-09.
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0263 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1102/1103 [============================>.] - ETA: 0s - loss: 0.0262 - s

In [ ]:
history2 = model.fit(x, epochs=3, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/3
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0262 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/3
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0264 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/3
1101/1103 [============================>.] - ETA: 0s - loss: 0.0263 - sparse_categorical_accuracy: 0.9995
Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.0479997697719911e-10.
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0263 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0259 - val_sparse_categorical_accuracy: 0.9992


In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0259 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0264 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1103/1103 [==============================] - ETA: 0s - loss: 0.0263 - sparse_categorical_accuracy: 0.9996
Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.095999650566285e-11.
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0263 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0261 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1102/1103 [============================>.] - ETA: 0s - loss: 0.0261 - sp

In [ ]:
history2 = model.fit(x, epochs=5, validation_data=(image_valid.reshape(image_valid.shape[0],60,60,1), np.array(label_valid)), callbacks=[reduce_lr,TerminateOnBaseline(monitor='val_accuracy', baseline=0.9997)],shuffle=True )

Epoch 1/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0262 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 2/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.0261 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 3/5
1103/1103 [==============================] - 23s 20ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9997 - val_loss: 0.0260 - val_sparse_categorical_accuracy: 0.9992
Epoch 4/5
1101/1103 [============================>.] - ETA: 0s - loss: 0.0263 - sparse_categorical_accuracy: 0.9996
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.6383998324709382e-12.
1103/1103 [==============================] - 23s 21ms/step - loss: 0.0263 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.0261 - val_sparse_categorical_accuracy: 0.9992
Epoch 5/5
1103/1103 [==============================] - 22s 20ms/step - loss: 0.02

## Model Evaluation

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 26ms/step - loss: 0.0367 - sparse_categorical_accuracy: 0.9975


[0.03665178641676903, 0.9974663257598877]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 26ms/step - loss: 0.0366 - sparse_categorical_accuracy: 0.9975


[0.03663952648639679, 0.9974663257598877]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 26ms/step - loss: 0.0367 - sparse_categorical_accuracy: 0.9975


[0.03669041767716408, 0.9975455403327942]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 27ms/step - loss: 0.0367 - sparse_categorical_accuracy: 0.9975


[0.03668085113167763, 0.9974663257598877]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 27ms/step - loss: 0.0367 - sparse_categorical_accuracy: 0.9975


[0.03670172393321991, 0.9974663257598877]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 27ms/step - loss: 0.0368 - sparse_categorical_accuracy: 0.9975


[0.0367855504155159, 0.9974663257598877]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 26ms/step - loss: 0.0366 - sparse_categorical_accuracy: 0.9974


[0.03663713485002518, 0.997387170791626]

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

50/50 [==============================] - 1s 28ms/step - loss: 0.0424 - sparse_categorical_accuracy: 0.9970


[0.04237156733870506, 0.9969912767410278]

## Visualization

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Model Summary 
Number of parameters used in the model architecture

In [ ]:
print(model.summary())

## Saving Trained .h5 Model

In [ ]:
def saveCNNModel(model_name):
    model.save(model_name)

In [ ]:
saveCNNModel('Trained_Models/winning_model_keras_augmentation_99_75_accuracy.h5')

In [ ]:
model._set_inputs(image_train,np.array(label_train))

## Loading Saved Trained .h5 Model

In [ ]:
from keras.models import load_model
def load_model_():
    Model = load_model('/content/drive/My Drive/Source_Code/Model/Trained_Models/winning_model_keras_augmentation_99_69.h5')
    return Model

In [ ]:
# del model
from tensorflow import keras

model= load_model_()

## Computation Avg. Processing Time

In [ ]:
import time
t1=time.time()
model.predict(test_image_array)
t2=time.time()
print("Average Processing time: ", ((t2-t1)/12630)*1000, " ms")

**bold text**

**Visualize**




In [ ]:
from keras import Model
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_layer(layer_name,image):
    # # check if the layer_name is correct 
    # assert layer_name in vgg_layers
    
    features_extractor = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    feature_maps = features_extractor.predict(image)[0]
    print("At layer \"{}\" : {} ".format(layer_name, feature_maps.shape))
    plot_feature_maps(feature_maps)

In [ ]:
# this function will plot all the feature maps within a fig of size (20,20)
def plot_feature_maps(feature_maps):
    height, width, depth = feature_maps.shape
    nb_plot = int(np.rint(np.sqrt(depth)))
    fig = plt.figure(figsize=(20, 20))
    for i in range(depth):
        plt.subplot(nb_plot, nb_plot, i+1)
        plt.imshow(feature_maps[:,:,i], cmap='gray')
        plt.title('feature map {}'.format(i+1))
    plt.show()

In [ ]:
plot_layer('conv2d_1',train_image_array[0].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[0].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_3',train_image_array[0].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_4',train_image_array[0].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_5',train_image_array[0].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_6',train_image_array[0].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[20].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[20].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_3',train_image_array[20].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_4',train_image_array[20].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_5',train_image_array[20].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_6',train_image_array[20].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[100].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[100].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_3',train_image_array[100].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[300].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[300].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[500].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1000].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1000].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1100].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1100].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1200].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1100].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1300].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1300].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1300].reshape(1,60,60,1))


In [ ]:
plt.imshow(train_image_array[1400].reshape(60,60))

In [ ]:
plot_layer('conv2d_1',train_image_array[1400].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1400].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1500].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1500].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1600].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1600].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1700].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1700].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1800].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1800].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[1900].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[1900].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_1',train_image_array[2000].reshape(1,60,60,1))


In [ ]:
plot_layer('conv2d_2',train_image_array[2000].reshape(1,60,60,1))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
np.random.seed(42)
# hist,bin_edges = np.histogram(train_image_labels)


# plt.bar(bin_edges[:-1], hist, width = 0.5, color='#0504aa',alpha=0.7)
# plt.xlim(min(bin_edges), max(bin_edges))
# plt.grid(axis='y', alpha=0.75)
# plt.xlabel('Class',fontsize=15)
# plt.ylabel('number of images per class',fontsize=15)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.ylabel('number of images per class',fontsize=15)
# plt.title('Class',fontsize=15)
# plt.show()

plt.figure(figsize=[10,8])
plt.hist(train_image_labels,density=False, bins=43)  # `density=False` would make counts
plt.ylabel('number of images per class')
plt.xlabel('class');

In [ ]:
file = open('/content/drive/My Drive/[ Masters ] - Deep Learning Proj/DataSet/Processed_DataSet/BelgiumTrainDataSet.pkl', 'rb')
train_image_array, train_image_labels = pickle.load(file)
file.close()

In [ ]:
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,15))

# N is the count in each bin, bins is the lower-limit of the bin
N, bins, patches = axs[0].hist(train_image_labels, bins=43)

# We'll color code by height, but you could use any scalar
fracs = N / N.max()

# we need to normalize the data to 0..1 for the full range of the colormap
norm = colors.Normalize(fracs.min(), fracs.max())

# Now, we'll loop through our objects and set the color of each accordingly
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)

# We can also normalize our inputs by the total number of counts
axs[1].hist(train_image_labels, bins=43, density=True)

# Now we format the y-axis to display percentage
axs[1].yaxis.set_major_formatter(PercentFormatter(xmax=1))